<a href="https://colab.research.google.com/github/JackeYou/Enjoy_LLM/blob/main/infer_local_gpu_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# gpu 本地部署
!git clone https://huggingface.co/THUDM/chatglm-6b

In [ ]:
%pwd
!git clone https://github.com/JackeYou/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt

In [ ]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import PeftModel
from tokenization_chatglm import ChatGLMTokenizer
from transformers import AutoTokenizer

# 加载本地预训练模型
torch.set_default_tensor_type(torch.cuda.HalfTensor)
BASE_MODEL_DIR = "/content/chatglm-6b"
gpu_model = ChatGLMForConditionalGeneration.from_pretrained(BASE_MODEL_DIR).half().cuda()
tokenizer = ChatGLMTokenizer.from_pretrained(BASE_MODEL_DIR)


In [ ]:
from cover_alpaca2jsonl import format_example

instructions = [
 {'instruction': '请给出十字路口交通指示灯的三种颜色',
  'input': '',
  'output': '红色、绿色、黄色。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '猪肉概念股大幅拉升，巨星农牧涨停，新五丰连续两日大涨',
  'output': '猪肉概念股，巨星农牧，新五丰。',
 },
 {'instruction': '以下哪家公司市值最高',
  'input': '腾讯、京东、淘宝',
  'output': '腾讯公司市值最高。截止2021年4月21日，腾讯公司的市值约为6.78万亿人民币，京东的市值约为1.26万亿人民币，淘宝的市值约为1.8万亿人民币。',
 },
 {'instruction': '请给出以下资讯3个关键词',
  'input': '隆基绿能董事长：光伏产品短期价格涨跌意义不大，长期看成本一定会越来越低',
  'output': '1.隆基绿能，2.光伏产品，3.成本降低',
 }
]
with torch.no_grad():
    for idx, item in enumerate(instructions):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.tensor([ids])
        out = gpu_model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"chatGPT### {idx+1}.Answer:\n", item.get('output'), '\n\n')